# chargement des bibliothèques et chemins

In [100]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from google.colab import files
import pandas as pd
import pytz
import os
import numpy as np
import re

timestamp = pd.Timestamp.now(tz='Europe/Paris').strftime('%Y%m%d_%H%M')

os.makedirs('/content/downloads', exist_ok=True)

data_path = '/content/drive/MyDrive/Elexxion/emploi/datas/FD_csv_EEC23.csv'
metadata_path = '/content/drive/MyDrive/Elexxion/emploi/metadatas/Varmod_EEC_2023.csv'
log_path = f'/content/downloads/transformation_log_{timestamp}.txt'
values_log_path = f'/content/downloads/unique_values_log_{timestamp}.txt'
base_raw_path = f'/content/downloads/base_raw_{timestamp}.parquet'
base_cleaned_path = f'/content/downloads/base_cleaned_{timestamp}.parquet'

# Contrôle sur l'année passée en paramétre
file_year_match = re.search(r'EEC(\d{2})', os.path.basename(data_path))
if file_year_match:
    file_year = int('20' + file_year_match.group(1))
else:
    file_year = None
    print("Le fichier doit être nommé avec EECXX, XX étant l'année des données")
    log_problems.append("Le fichier doit être nommé avec EECXX, XX étant l'année des données")

metadata = pd.read_csv(metadata_path, sep=';', encoding='utf-8')

Mounted at /content/drive


# mapping metadata

In [102]:
# Remplace les COD_MOD vides par np.nan pour uniformiser les clés
metadata['COD_MOD'] = metadata['COD_MOD'].replace('', np.nan)

# Crée les mappings
col_code_to_label = (
    metadata.groupby('COD_VAR')['LIB_VAR']
    .agg(lambda x: x.value_counts().index[0] if not x.value_counts().empty else None)
    .to_dict()
)

# Construire value_mapping en traitant ANNEE à part
value_mapping = {}
for idx, row in metadata.iterrows():
    cod_var = row['COD_VAR']
    cod_mod = row['COD_MOD']
    lib_mod = row['LIB_MOD']

    if cod_var == 'ANNEE' and pd.notnull(cod_mod):
        value_mapping[(cod_var, cod_mod)] = cod_mod
    else:
        value_mapping[(cod_var, cod_mod)] = lib_mod

column_constraints = (
    metadata.groupby('COD_VAR')[['TYPE_VAR', 'LONG_VAR']]
    .first()
    .to_dict(orient='index')
)

print("[('code', \"label\")] -> ", list(col_code_to_label.items())[:5])
print("[(('COD_VAR', 'COD_MOD'), 'LIB_MOD') ->", list(value_mapping.items())[:5])
print("column_constraints pour les contraintes :", list(column_constraints.items())[:5])

codvar_null_or_empty = metadata[metadata['COD_VAR'].isnull() | (metadata['COD_VAR'].astype(str).str.strip() == '')]

# si des lignes sont trouvées
if not codvar_null_or_empty.empty:
    print(f"{len(codvar_null_or_empty)} ligne(s) avec COD_VAR vide ou null détectée(s)")
    print(codvar_null_or_empty.head(3))

[('code', "label")] ->  [('AAC', "Exercice d'une activité professionnelle régulière antérieure, pour les inactifs, chômeurs et personnes ayant une activité temporaire ou d'appoint autre qu'un emploi informel"), ('ACL_EMPLOI', "Classe d'emploi (emploi actuel ou dernier emploi)"), ('ACTEU', "Statut d'activité au sens du Bureau International du Travail (BIT)"), ('AGE6', None), ('AISCO2', 'Code ISCO (dernier emploi, niveau 2)')]
[(('COD_VAR', 'COD_MOD'), 'LIB_MOD') -> [(('AAC', '1'), 'Oui'), (('AAC', '2'), 'Non'), (('AAC', '9'), 'Non réponse'), (('AAC', nan), 'Hors champ'), (('ACL_EMPLOI', 'I1'), 'Emploi indépendant de niveau supérieur')]
column_constraints pour les contraintes : [('AAC', {'TYPE_VAR': 'CHAR', 'LONG_VAR': 1.0}), ('ACL_EMPLOI', {'TYPE_VAR': 'CHAR', 'LONG_VAR': 3.0}), ('ACTEU', {'TYPE_VAR': 'CHAR', 'LONG_VAR': 1.0}), ('AGE6', {'TYPE_VAR': 'CHAR', 'LONG_VAR': 2.0}), ('AISCO2', {'TYPE_VAR': 'CHAR', 'LONG_VAR': 4.0})]


# chargement des données par chunk et transformation

In [103]:
# Lecture des données par chunks
chunksize = 100_000
data = pd.read_csv(data_path, sep=';', encoding='utf-8', chunksize=chunksize)

transformed_chunks = []
log_problems = []
unique_values = {}

for chunk_idx, chunk in enumerate(data):
    print(f'Traitement du chunk {chunk_idx + 1} → {chunk.shape[0]} lignes, {chunk.shape[1]} colonnes')

    # Suppression des lignes vides
    chunk.dropna(how='all', inplace=True)
    if chunk.empty:
        print(f"Chunk {chunk_idx + 1} vide après suppression des lignes nulles, ignoré.")
        continue

    # Renommer les colonnes COD_VAR -> LIB_VAR
    col_rename = {col: col_code_to_label.get(col, col) for col in chunk.columns}
    chunk.rename(columns=col_rename, inplace=True)

    chunk_transformed = chunk.copy()

    if 'ANNEE' in chunk_transformed.columns and file_year:
        # Forcer au format entier
        chunk_transformed['ANNEE'] = pd.to_numeric(chunk_transformed['ANNEE'], errors='coerce').astype('Int64')

        # Contrôle sur l'année
        invalid_years = ~chunk_transformed['ANNEE'].between(2000, 2100)
        chunk_transformed.loc[invalid_years, 'ANNEE'] = np.nan
        chunk_transformed['ANNEE'].fillna(file_year, inplace=True)
        chunk_transformed['ANNEE'] = chunk_transformed['ANNEE'].astype(int)
        print(f"Colonne ANNEE contrôlée et corrigée avec l'année {file_year} issue du fichier.")

    for col in chunk_transformed.columns:
        # Trouver le COD_VAR correspondant
        cod_var = next((k for k, v in col_code_to_label.items() if v == col), None)
        if cod_var is None:
            print(f"Colonne sans mapping COD_VAR (ignorée) : {col}")
            continue

        #col_lib = col
        col_values = chunk_transformed[col].unique()
        unique_values.setdefault(col, set()).update(col_values)

        # Supprimé : contrôle des contraintes type_var, long_var

        def map_value(val):
            if pd.isnull(val) or str(val).strip() == '':
                return str(value_mapping.get((cod_var, float('nan')), 'Hors champ'))
            return str(value_mapping.get((cod_var, str(val)), val))

        chunk_transformed[col] = chunk_transformed[col].apply(map_value)

    transformed_chunks.append(chunk_transformed)

# Fusionner les chunks
final_df = pd.concat(transformed_chunks, ignore_index=True)

Traitement du chunk 1 → 100000 lignes, 83 colonnes
Traitement du chunk 2 → 100000 lignes, 83 colonnes
Traitement du chunk 3 → 100000 lignes, 83 colonnes
Traitement du chunk 4 → 48624 lignes, 83 colonnes


# fusion des chunks et enregistrement en parquet

In [105]:
# Fusionner les chunks
final_df = pd.concat(transformed_chunks, ignore_index=True)

# Forcer les noms de colonnes en chaînes de caractères pour éviter le warning
final_df.columns = final_df.columns.map(str)

# Sauvegarde en Parquet
try:
    final_df.to_parquet(base_raw_path)
    print(f'Transformation terminée et sauvegardée sous {base_raw_path}')
except Exception as e:
    print(f'Erreur lors de la sauvegarde Parquet : {e}')

# Aperçu du Parquet
try:
    parquet_sample = pd.read_parquet(base_raw_path)
    print("Aperçu de la première ligne du Parquet :")
    print(parquet_sample.head(1))
except Exception as e:
    print(f'Erreur lors de la lecture du Parquet : {e}')

Transformation terminée et sauvegardée sous /content/downloads/base_raw_20250504_1937.parquet
Aperçu de la première ligne du Parquet :
  Exercice d'une activité professionnelle régulière antérieure, pour les inactifs, chômeurs et personnes ayant une activité temporaire ou d'appoint autre qu'un emploi informel  \
0                                                1.0                                                                                                                              

   Classe d'emploi (emploi actuel ou dernier emploi)  \
0  Emploi salarié qualifié, d'orientation employé...   

  Statut d'activité au sens du Bureau International du Travail (BIT)  \
0                                     Non Inactivité                   

        None Code ISCO (dernier emploi, niveau 2) Ancienneté au chômage  \
0  65-89 ans                                 33.0            Hors champ   

  Ancienneté en 4 tranches (emploi principal) Ancienneté sans emploi  \
0                        

# sauvegarde des logs

In [110]:
# Logs des problèmes rencontrés
with open(log_path, 'w') as log_file:
    for problem in log_problems:
        log_file.write(problem + '\n')
print(f'Log des problèmes écrit dans {log_path}')

# Logs des valeurs distinctes par colonne
with open(values_log_path, 'w') as val_file:
    for col, values in unique_values.items():
        sorted_values = sorted([str(v) for v in values])
        val_file.write(f'{col} → {sorted_values}\n')
print(f'Log des valeurs distinctes écrit dans {values_log_path}')

Log des problèmes écrit dans /content/downloads/transformation_log_20250504_1937.txt
Log des valeurs distinctes écrit dans /content/downloads/unique_values_log_20250504_1937.txt


# transformation format base_cleaned.parquet

In [117]:
# Shape initial
print("Shape avant nettoyage initial :", final_df.shape)

# Supprimer colonnes totalement vides
empty_cols = final_df.columns[final_df.isnull().all()]
df_raw_cleaned = final_df.drop(columns=empty_cols)
print(f"Colonnes totalement vides supprimées : {len(empty_cols)} → {list(empty_cols)}")

# Supprimer colonnes constantes (une seule valeur unique)
# TODO récupéré ANNEE et non la valeur du dataframe
constant_cols = [col for col in df_raw_cleaned.columns if col != 'ANNEE' and df_raw_cleaned[col].nunique() == 1]
df_raw_cleaned = df_raw_cleaned.drop(columns=constant_cols)
print(f"Colonnes constantes supprimées (hors ANNEE) : {len(constant_cols)} → {list(constant_cols)}")

# Vérifier les valeurs uniques de ANNEE
if 'ANNEE' in df_raw_cleaned.columns:
    print(f"Valeurs uniques de ANNEE : {df_raw_cleaned['ANNEE'].unique()}")

# (Optionnel) Supprimer colonnes avec trop de NaN (> 90%)
nan_threshold = 0.9
nan_cols = df_raw_cleaned.columns[df_raw_cleaned.isnull().mean() > nan_threshold]
df_raw_cleaned = df_raw_cleaned.drop(columns=nan_cols)
print(f"Colonnes avec plus de 90% de NaN supprimées : {len(nan_cols)} → {list(nan_cols)}")

# Shape après nettoyage initial
print("Shape après nettoyage initial :", df_raw_cleaned.shape)

Shape avant nettoyage initial : (348624, 83)
Colonnes totalement vides supprimées : 0 → []
Colonnes constantes supprimées (hors ANNEE) : 1 → ["Année de référence de l'enquête"]
Colonnes avec plus de 90% de NaN supprimées : 0 → []
Shape après nettoyage initial : (348624, 82)


# suppression des données aberrantes

In [118]:
import numpy as np

# Copier pour éviter d’écraser
df_cleaned = df_raw_cleaned.copy()

# Identifier les colonnes numériques (d’après metadata ou heuristique)
num_cols = [col for col in df_cleaned.columns if col != 'ANNEE' and pd.api.types.is_numeric_dtype(df_cleaned[col])]

# Convertir les colonnes numériques (forcer au besoin)
for col in num_cols:
    df_cleaned[col] = pd.to_numeric(df_cleaned[col], errors='coerce')

# Remplacer les valeurs infinies par NaN (LightGBM n’aime pas les infs)
df_cleaned.replace([np.inf, -np.inf], np.nan, inplace=True)

# Afficher le nombre de NaN par colonne
nan_summary = df_cleaned.isnull().sum()
print("Nombre de NaN par colonne :")
print(nan_summary[nan_summary > 0])

# Décider quoi faire avec les NaN :
# ici exemple simple → remplir les colonnes numériques par la médiane
for col in num_cols:
    median_value = df_cleaned[col].median()
    df_cleaned[col].fillna(median_value, inplace=True)

# Vérifier à nouveau les NaN
print("Nombre de NaN après remplissage :")
print(df_cleaned.isnull().sum().sum())

# Sauvegarder le fichier nettoyé
df_cleaned.to_parquet(base_cleaned_path)
print(f"Base nettoyée sauvegardée sous : {base_cleaned_path}")

Nombre de NaN par colonne :
Series([], dtype: int64)
Nombre de NaN après remplissage :
0
Base nettoyée sauvegardée sous : /content/downloads/base_cleaned_20250504_1937.parquet


# téléchargement local si besoin

In [ ]:
files.download(base_raw_path)

files.download(log_path)

files.download(values_log_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# début modèle LM

In [ ]:
import lightgbm as lgb

# Créer un dataset LightGBM
train_data = lgb.Dataset(X, label=y, categorical_feature=categorical_columns)

# Paramètres LightGBM (tu peux ajuster ces paramètres en fonction de ton problème)
params = {
    'objective': 'binary',  # pour une classification binaire
    'metric': 'binary_error',  # métrique à utiliser
    'boosting_type': 'gbdt',
    'num_leaves': 31,  # tu peux ajuster ces paramètres
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
}

# Entraînement du modèle
clf = lgb.train(params, train_data, 100)